# Part I Python Fundamentals 2

This is my review note of Python for the purpose of self-study. The note mixes up with English & Chinese.
- Part I follows the *[Liao's Python tutorial](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000) (in Chinese)*

In [2]:
# Display multiple interactive objects in one shell
# No Need for print function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 4.函数式编程 Functional Programming

- 对应到编程语言，就是越低级的语言，越贴近计算机，抽象程度低，执行效率高，比如C语言；越高级的语言，越贴近计算，抽象程度高，执行效率低，比如Lisp语言。函数式编程就是一种抽象程度很高的编程范式
- 纯粹的函数式编程语言编写的函数没有变量，因此，任意一个函数，只要输入是确定的，输出就是确定的，这种纯函数我们称之为没有副作用。而允许使用变量的程序设计语言，由于函数内部的变量状态不确定，同样的输入，可能得到不同的输出，因此，这种函数是有副作用的。
- **函数式编程的一个特点就是，允许把函数本身作为参数传入另一个函数，还允许返回一个函数!**

高阶函数 Higher-order Function

- 变量指向函数

In [108]:
x = abs
x

<function abs>

In [110]:
# change built-in function abs point to 2
abs = 2
abs(-3)

# 要恢复abs函数，请重启Python交互环境

TypeError: 'int' object is not callable

In [ ]:
from math import sqrt
def add(x, y, f):
    return f(x) + f(y)

add(4, 9, sqrt)

Map/Reduce

我们先看map。map()函数接收两个参数，一个是函数，一个是Iterable，map将传入的函数依次作用到序列的每个元素，并把结果作为新的Iterator返回

In [ ]:
def f(s):
    return s.lower()
r = map(f, ['A', 'B', 'C', 'D']) # Iterator
list(r)

map()作为高阶函数，事实上它把运算规则抽象了

In [ ]:
# 抽象化,只需要一行代码
list(map(str, [1, 2, 3]))

再看reduce的用法。reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算其效果就是：

`reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)`

In [4]:
# 把序列[1, 3, 5, 7, 9]变换成整数13579
from functools import reduce
def f1(x, y):
    return x + y

# Fucking cool
int(reduce(f1, list(map(str, [1, 3, 5, 7, 9]))))
int(reduce(f1, tuple(map(str, [1, 3, 5, 7, 9])))) # tuple 作为序列 同样也可以

13579

13579

In [ ]:
# 另一种更简单的实现
def f2(x, y):
    return 10 * x + y
reduce(f2, [1, 3, 5, 7, 9])

以下，用 map/reduce 重写 int()/str()

In [5]:
from functools import reduce

# 不会改变的常量，用大写
DIGITS = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

def char2num(s):
    return DIGITS[s]

# 一行 lambda 解决问题 Fucking cool
def str2int(s):
    return reduce(lambda x, y: x * 10 + y, map(char2num, s))

利用map()函数，把用户输入的不规范的英文名字，变为首字母大写，其他小写的规范名字。输入：['adam', 'LISA', 'barT']，输出：['Adam', 'Lisa', 'Bart']：

In [ ]:
def name(lst):
    return list(map(lambda s: s[0].upper() + s[1:].lower(), lst))
name(['adam', 'LISA'])

写一个prod()函数，可以接受一个list并利用reduce()求积：

In [ ]:
from functools import reduce
def prod(lst):
    return reduce(lambda a, b: a * b, lst)
prod([1, 2, 3])

利用map和reduce编写一个str2float函数，把字符串'123.456'转换成浮点数123.456：

In [ ]:
# Fucking cool
def str2float(s):
    nbs = s.split('.')
    return reduce(lambda a, b: a * 10 + b, map(int, nbs[0])) + \
           reduce(lambda a, b: a * 10 + b, map(int, nbs[1])) / 10**len(nbs[1])

str2float('123.456')

Filter()
- filter 返回的是一个 generator

和map()类似，filter()也接收一个函数和一个序列。和map()不同的是，filter()把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

In [111]:
# filter function
# must return T or F
def not_empty(s):
    return s and s.strip()

list(filter(not_empty, ['A', '', 'B', None, 'C', '  ']))

['A', 'B', 'C']

注意到filter()函数返回的是一个Iterator，也就是一个惰性序列，所以要强迫filter()完成计算结果，需要用list()函数获得所有结果并返回lis

用埃氏筛法产生一个 prime number generator

In [6]:
# odd numbers generator
# 3, 5, 7, 9 ...
def odd_gen():
    n = 1
    while True:
        n += 2
        yield n

# filter
def not_divisible(n):
    return lambda x: x % n > 0

# primes
def primes():
    yield 2
    nbs = odd_gen()  # 对于 generatore, 一定一定要构造一个新的序列, 才能进行修改
    while True:
        nb = next(nbs)
        yield nb
        nbs = filter(not_divisible(nb), nbs)  # 传入filter的一定是一个序列, 还要构造一个新的序列

for n in primes():
    if n < 100:
        print(n, end=' ')
    else:
        break # 一定一定要加 break 不然 python 会继续一直计算，但不会输出

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 

**注意 call primes() 时候的 break, 一定要加上，不然 kernel 无限产生序列**

Iterator是惰性计算的序列，所以我们可以用Python表示“全体自然数”，“全体素数”这样的序列，而代码非常简洁

In [20]:
# 请利用filter()筛选出回数：
# 回数是指从左向右读和从右向左读都是一样的数，例如12321，909
def x_x(s):
    s = str(s)
    half_l = len(s) // 2
    for i in range(half_l):
        if s[i] == s[-(i + 1)]:
            continue
        else:
            return False
    return True

list(filter(x_x, range(1, 100)))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 22, 33, 44, 55, 66, 77, 88, 99]

In [7]:
# 一个更好的实现
# Fucking cool
def x_x(s):
    s = str(s)
    return s == s[::-1]

list(filter(x_x, range(1, 100)))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 22, 33, 44, 55, 66, 77, 88, 99]

Sorted

`sorted(list, key)`也是一个高阶函数, **key 其实就是一个 map 函数**

In [26]:
# key 实际作用就是 map, 先 abs(element) 每一个 element，然后排序，然后按照排序 返回 原element
sorted([36, 5, -12, 9, -21], key=abs)

[5, 9, -12, -21, 36]

所以可以自定义 key function

In [27]:
def divisor_three(n):
    return n % 3 
sorted([36, 5, -12, 9, -21], key=divisor_three)

[36, -12, 9, -21, 5]

函数作为返回值

In [34]:
def lazy_sum(*args): # 返回函数 （会被立即执行）
    def sum(): # 内部函数 （不会被立即执行）
        ax = 0
        for n in args: # 闭包，用了 lazy_sum 的 parameter
            ax = ax + n
        return ax
    return sum

f1 = lazy_sum(1, 2, 3, 4, 5) # 此时并没有求和
f1() # 此时才求和

15

闭包 Closure

「函数」和「函数内部能访问到的变量」的总和，就是一个闭包

- 为什么要函数套函数呢？

是因为需要局部变量，所以才把 local 放在一个函数里，如果不把 local 放在一个函数里，local 就是一个全局变量了，达不到使用闭包的目的——隐藏变量
### 所以函数套函数只是为了造出一个局部变量，跟闭包无关。
#### return inner_function 也只是为了 inner_function 可以被使用，与闭包无关。

- 闭包的作用

隐藏变量 （替代全局变量），用来保存一个需要持久保存的变量

**返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。**

In [51]:
# 错误例子
def count(): # 返回函数 （会被立即执行）
    fs = [] # 返回函数 （会被立即执行）
    for i in range(1, 4): # 返回函数 （会被立即执行）！！
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count() # i = 3
f1(), f2(), f3() 

(9, 9, 9)

In [8]:
# 正确例子 真JB麻烦
# 如果一定要引用循环变量怎么办？方法是再创建一个函数，用该函数的参数绑定循环变量当前的值
# 以下标明 运行步骤
def count(): # 1
    def f(x): # 6
        def g(): # 8
            return x * x # 9
        return g # 7
    fs = [] # 3
    for i in range(1, 4): # 4
        fs.append(f(i)) # 5
    return fs # 2

f1, f2, f3 = count()
f1(), f2(), f3()

(1, 4, 9)

In [9]:
# 利用闭包返回一个 generator 计数器函数，每次调用它返回递增整数：
def createCounter():
    i = 0
    def f():
        nonlocal i  # 因为 i 为不可变的
        i += 1
        return i
    return f

counter = createCounter()
counter()
counter()
counter()

1

2

3

**重要：闭包引用的外部变量如果是不可变的，且需要在返回的函数中修改，则要在函数中用nonlocal声明。** (为什么呢，暂时没想明白）

Lambda

- 关键字lambda表示匿名函数，冒号前面的x表示函数参数
- 匿名函数有个限制，就是只能有一个表达式，不用写return，
### 返回值就是该表达式的结果

In [10]:
# lambda 函数和普通函数一样，可以被起名字
f = lambda x: x * x
f

<function __main__.<lambda>>

Decorator 装饰器
- decorator可以增强函数的功能，定义起来虽然有点复杂，但使用起来非常灵活和方便

In [71]:
def log(func):
    def wrapper(*args, **kw):
        print('calling %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

@log
def now():
    print('2015-3-25')
now()

calling now():
2015-3-25


- 首先运行 log(now()) 
- 因为 return wrapper 
- 所以运行 wrapper()

但是注意 now() 的函数名已经被改为 wrapper(), 因为返回的那个wrapper()函数名字就是'wrapper'，所以，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。

In [75]:
now.__name__

'wrapper'

这时候需要**用 functools.wrapper**

In [12]:
# 会把原函数 now() 的一些属性 复制到 return 里面， 不会出错。
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

@log
def now():
    print('2015-3-25')
now.__name__

'now'

In [13]:
# 请设计一个decorator，它可作用于任何函数上，并打印该函数的执行时间：
import time, functools

def metric(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kwargs):
        before = time.time()
        f = fn(*args, **kwargs)
        after = time.time()
        t = after - before
        print('%s executed in %s ms' % (fn.__name__, t))
        return f
    return wrapper

@metric
def f():
    print('hi')

f()

hi
f executed in 3.0040740966796875e-05 ms


偏函数
- `from functools import partial`
- 作用是固定函数中的参数

In [14]:
sorted([5, -6, 1])
sorted([5, -6, 1], key=abs)
# 如果我们想一直保留 key=abs

[-6, 1, 5]

[1, 5, -6]

In [96]:
from functools import partial
abs_sort = partial(sorted, key=abs)
abs_sort([5, 6, -1])
# 保留成功

[-1, 5, 6]

In [101]:
# 但如果这样
max2 = functools.partial(max, 4)
max2(1, 2, 3)
# 原因在于 max 参数接受 *args, 4 被认为是 和1， 2， 3 一样的参数输入进去了

4

### 5.模块
- 在Python中，一个.py文件就称之为一个模块（Module）
- Package 包换很多.py文件
- 请注意，每一个包目录下面都会有一个`__init__.py`的文件，
### 这个文件是必须存在的，否则，Python就把这个目录当成普通目录，而不是一个包。
`__init__.py`可以是空文件，也可以有Python代码，因为`__init__.py`本身就是一个模块，它的名字就是Package的名字
- 所以可以出现多层Package，如下

#### Python 标准格式

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

' a test module '

__author__ = 'Michael Liao'

import sys

def test():
    args = sys.argv # <STDIN>
    if len(args)==1:
        print('Hello, world!')
    elif len(args)==2:
        print('Hello, %s!' % args[1])
    else:
        print('Too many arguments!')

if __name__=='__main__': # module test
    test()

' a test module '

Too many arguments!


作用域
- 特殊 变量/函数/方法  `__init__`
- private 变量/函数/方法 `_var` / `_function`
- 防重名 变量/函数/方法 `__var`

之所以我们说，private函数和变量“不应该”被直接引用，而不是“不能”被直接引用，是因为Python并没有一种方法可以完全限制访问private函数或变量，但是，从编程习惯上不应该引用private函数或变量

In [17]:
# _private_1 / 2 只是为了内部调用给 greeting（）
# 所以隐藏起来
def _private_1(name):
    return 'Hello, %s' % name

def _private_2(name):
    return 'Hi, %s' % name

def greeting(name):
    if len(name) > 3:
        return _private_1(name)
    else:
        return _private_2(name)


我们在模块里公开greeting()函数，而把内部逻辑用private函数隐藏起来了，这样，调用greeting()函数不用关心内部的private函数细节，这也是一种非常有用的代码封装和抽象的方法，即：

### 外部不需要引用的函数全部定义成private，只有外部需要引用的函数才定义为public。**

模块搜索路径

In [18]:
import sys
sys.path

# 添加路径
# sys.path.append('path')

['',
 '/Users/zhengtingli/anaconda/lib/python35.zip',
 '/Users/zhengtingli/anaconda/lib/python3.5',
 '/Users/zhengtingli/anaconda/lib/python3.5/plat-darwin',
 '/Users/zhengtingli/anaconda/lib/python3.5/lib-dynload',
 '/Users/zhengtingli/anaconda/lib/python3.5/site-packages',
 '/Users/zhengtingli/anaconda/lib/python3.5/site-packages/Sphinx-1.4.1-py3.5.egg',
 '/Users/zhengtingli/anaconda/lib/python3.5/site-packages/aeosa',
 '/Users/zhengtingli/anaconda/lib/python3.5/site-packages/setuptools-23.0.0-py3.5.egg',
 '/Users/zhengtingli/anaconda/lib/python3.5/site-packages/IPython/extensions',
 '/Users/zhengtingli/.ipython']